AGUDELO Julian et PONCE Afonso - 2023
# TD - Site web de rencontre de proteins

A continuation, nous allons nous consacrer à la conception d'un site web fondé sur l'ontologie GO, une ontologie très utilisée en bio-informatique.

### Sur Go :
L'ontologie GO se compose de trois parties: les processus biologiques, les fonctions moléculaires, et les composants cellulaires.
La troisième composante se concentre sur la description des éléments constitutifs d'une cellule, tels que les membranes, le noyau, et les organites.
La gestion de cette troisième partie s'avère complexe en raison de la coexistence de deux types de hiérarchies. D'une part, une hiérarchie d'héritage classique avec des relations "est-un" caractérise la structure. D'autre part, une hiérarchie basée sur la relation "partie-de", connue sous le nom de méronymie, vise à décomposer la cellule en sous-parties, puis en sous-sous-parties, avec la cellule entière en tant que racine et les parties indivisibles en tant que feuilles.

### Sur les fonctionnalités attendues du site web :
Ce site offrira la possibilité d'entrer deux noms de protéines et de déterminer les compartiments cellulaires dans lesquels elles pourraient éventuellement interagir. D'un point de vue biologique, cette fonctionnalité revêt une importance particulière, car des protéines dépourvues d'un "site de rencontre" commun ne peuvent pas interagir entre elles.

In [1]:
# Commençons par importer les bibliothèques nécessaires
from flask import Flask
from owlready2 import *
import mygene

Owlready2 stocke tous les triples dans un objet 'World', et peut gérer plusieurs mondes en parallèle. 'default_world' est le monde utilisé par défaut.

Owlready2 utilise un quadstore optimisé (comme les [Triplestore ou RDF store](https://en.wikipedia.org/wiki/Triplestore) , mais avec 4 composants) . Par défaut, le quadstore est stocké en mémoire, mais il peut également être stocké dans un fichier SQLite3. Cela permet la persistance.

In [ ]:

# La méthode .set_backend() de World définit le nom de fichier SQLite3 associé au quadstore, par exemple :
default_world.set_backend(filename = "quadstore.sqlite3")

# Une nouvelle ontologie vide peut être obtenue avec la fonction get_ontology() ; 
# elle prend un seul paramètre, l'IRI de l'ontologie. L'IRI est une sorte d'URL ; 
# les IRI sont utilisés comme identifiants pour les ontologies.
go = get_ontology("http://purl.obolibrary.org/obo/go.owl#").load()

# EXPLIQUER LA PARTIE NAMESPACES https://owlready2.readthedocs.io/en/latest/namespace.html?highlight=get_namespace#namespaces
obo = go.get_namespace("http://purl.obolibrary.org/obo/")
default_world.save()